# Case Study 1: Danny's Diner
Each of the following case study questions can be answered using a single SQL statement:

In [1]:
%load_ext sql
%sql sqlite:///dannys_diner.db

# Question 1. What is the total amount each customer spent at the restaurant?

In [2]:
%%sql

SELECT sales.customer_id, SUM(menu.price) as total_spent
FROM sales
LEFT JOIN menu ON sales.product_id = menu.product_id 
GROUP BY sales.customer_id

 * sqlite:///dannys_diner.db
Done.


customer_id,total_spent
A,76
B,74
C,36


# Question 2. How many days has each customer visited the restaurant?

In [3]:
%%sql

SELECT customer_id, COUNT(DISTINCT(order_date)) as num_days_visited
FROM sales
GROUP BY customer_id

 * sqlite:///dannys_diner.db
Done.


customer_id,num_days_visited
A,4
B,6
C,2


# Question 3. What was the first item from the menu purchased by each customer?

In [4]:
%%sql

WITH row_number AS (
    SELECT *,
        ROW_NUMBER() OVER(PARTITION BY customer_id) AS row_num
    FROM sales
    GROUP BY customer_id
)
SELECT row_number.customer_id, row_number.product_id, menu.product_name
FROM row_number
LEFT JOIN menu ON row_number.product_id = menu.product_id
WHERE row_num = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_id,product_name
A,1,sushi
B,2,curry
C,3,ramen


# Question 4. What is the most purchased item on the menu and how many times was it purchased by all customers?

In [5]:
%%sql

SELECT customer_id, COUNT(*) as count
FROM sales
WHERE product_id = (
    SELECT sales.product_id
    FROM sales
    LEFT JOIN menu ON sales.product_id = menu.product_id
    GROUP BY product_name
    ORDER BY COUNT(sales.product_id) DESC
    LIMIT 1
    )
GROUP BY customer_id;

 * sqlite:///dannys_diner.db
Done.


customer_id,count
A,3
B,2
C,3


# Question 5. Which item was the most popular for each customer?

In [6]:
%%sql

WITH row_number AS (
    SELECT customer_id, product_id, COUNT(*), ROW_NUMBER() OVER(PARTITION BY customer_id ORDER BY customer_id ASC, COUNT(*) DESC) AS row_num
    FROM sales
    GROUP BY customer_id, product_id
    ORDER BY customer_id ASC, COUNT(*) DESC
)
SELECT *
FROM row_number
WHERE row_num = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_id,COUNT(*),row_num
A,3,3,1
B,1,2,1
C,3,3,1


In [7]:
## ??do I need to merge menu and have the product name?

# Question 6. Which item was purchased first by the customer after they became a member?

In [8]:
# %%sql
# SELECT *
#     FROM sales
#     LEFT JOIN members ON sales.customer_id = members.customer_id
#     LEFT JOIN menu ON sales.product_id = menu.product_id
#     WHERE members.join_date < sales.order_date

In [9]:
%%sql

WITH row_number AS (
 SELECT sales.customer_id, sales.product_id, menu.product_name, ROW_NUMBER() OVER(PARTITION BY sales.customer_id) as row_num
    FROM sales
    LEFT JOIN members ON sales.customer_id = members.customer_id
    LEFT JOIN menu ON sales.product_id = menu.product_id
    WHERE members.join_date < sales.order_date
)

SELECT customer_id, product_id, product_name
FROM row_number
WHERE row_num = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_id,product_name
A,3,ramen
B,1,sushi


# Question 7. Which item was purchased just before the customer became a member?

In [10]:
%%sql
SELECT *
    FROM sales
    LEFT JOIN members ON sales.customer_id = members.customer_id
    LEFT JOIN menu ON sales.product_id = menu.product_id
    WHERE members.join_date > sales.order_date
    ORDER BY sales.customer_id, sales.order_date DESC

 * sqlite:///dannys_diner.db
Done.


customer_id,order_date,product_id,customer_id_1,join_date,product_id_1,product_name,price
A,2021-01-01,1,A,2021-01-07,1,sushi,10
A,2021-01-01,2,A,2021-01-07,2,curry,15
B,2021-01-04,1,B,2021-01-09,1,sushi,10
B,2021-01-02,2,B,2021-01-09,2,curry,15
B,2021-01-01,2,B,2021-01-09,2,curry,15


In [11]:
%%sql

WITH row_number AS (
    SELECT sales.customer_id, sales.product_id, menu.product_name, 
        ROW_NUMBER() OVER(PARTITION BY sales.customer_id  ORDER BY sales.customer_id, sales.order_date DESC) as row_num
    FROM sales
    LEFT JOIN members ON sales.customer_id = members.customer_id
    LEFT JOIN menu ON sales.product_id = menu.product_id
    WHERE members.join_date > sales.order_date
)
SELECT customer_id, product_id, product_name
FROM row_number
WHERE row_num = 1;

 * sqlite:///dannys_diner.db
Done.


customer_id,product_id,product_name
A,1,sushi
B,1,sushi


# Question 8. What is the total items and amount spent for each member before they became a member?

In [13]:
# %%sql


# Question 9. If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have? In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?

In [ ]:
%%sql


## Bonus Questions

## Join All The Things
The following questions are related creating basic data tables that Danny and his team can use to quickly derive insights without needing to join the underlying tables using SQL.

In [ ]:
%%sql


# Rank All The Things
Danny also requires further information about the ranking of customer products, but he purposely does not need the ranking for non-member purchases so he expects null ranking values for the records when customers are not yet part of the loyalty program.

In [ ]:
%%sql
